In [14]:
import os
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from PIL import Image, ImageTk

# Step 1: Read the folder
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            img = cv2.imread(os.path.join(folder_path, filename))
            if img is not None:
                images.append((filename, img))
    return images

if __name__ == "__main__":
    folder_path = "C:\\Users\\ravik\\Desktop\\My images\\test"
    
    # Step 1: Read the folder
    images = load_images_from_folder(folder_path)
    
    
    

In [77]:

# Step 2: Process images and collect face data
def detect_faces(images):
    face_data = []
    #face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    face_cascade = cv2.CascadeClassifier('C:\\Users\\ravik\\Documents\\GitHub\\opencv\\data\\haarcascades\\haarcascade_frontalface_default.xml')
    tempnum = 0
    for filename, img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]    
            resized_face = cv2.resize(face, (180, 180))  # Define new_width and new_height   
            tempnum = tempnum+1
            faceId = "face"+str(tempnum)
            face_data.append((faceId, filename, resized_face))
    return face_data
# Step 2: Process images and collect face data
face_data = detect_faces(images)
# Create a DataFrame to store face data
df = pd.DataFrame(data={
    'FaceId': [faceId for faceId, _, _ in face_data],
    'Filename': [filename for _, filename, _ in face_data],
    'Face': [face for _, _, face in face_data],
    'Consider': [0] * len(face_data)
})
df
    

,FaceId,Filename,Face,Consider
0,face1,20161006_222700.jpg,"[[88, 87, 83, 81, 80, 85, 85, 77, 84, 80, 78, ...",0
1,face2,20161007_203010.jpg,"[[254, 254, 250, 254, 252, 254, 243, 210, 157,...",0
2,face3,20161007_203010.jpg,"[[185, 184, 186, 186, 186, 186, 185, 185, 184,...",0
3,face4,20161007_203010.jpg,"[[174, 174, 174, 175, 175, 175, 175, 175, 175,...",0
4,face5,20161007_203010.jpg,"[[111, 87, 80, 76, 65, 64, 54, 46, 37, 31, 33,...",0
5,face6,20161008_021946.jpg,"[[191, 186, 181, 99, 35, 41, 47, 50, 48, 48, 5...",0
6,face7,20161008_021946.jpg,"[[24, 26, 28, 29, 30, 29, 32, 33, 35, 36, 39, ...",0


In [78]:
# Step 3: Ask the user which faces to consider
def user_face_selection(face_data):
    user_choices = {}
    for filename, faces in face_data:
        
        #for face in faces:
           # print(face)
            #image1 = Image.fromarray(face)
        cv2.imshow('Face', faces)
        cv2.waitKey()
        user_input = input(f"Is the face in {filename} important? (y/n): ").lower()
        if user_input == 'y':
            user_choices[filename] = 1
        else:
            user_choices[filename] = 0
    return user_choices
# Step 3: Ask the user which faces to consider
user_choices = user_face_selection(face_data)

ValueError: too many values to unpack (expected 2)

In [28]:
face_data[0][1]

array([[ 96,  80,  88, ..., 238, 238, 238],
       [ 90,  85,  89, ..., 238, 238, 238],
       [ 87,  86,  90, ..., 238, 238, 238],
       ...,
       [ 63,  61,  59, ..., 251, 254, 242],
       [ 64,  60,  57, ..., 217, 254, 247],
       [ 65,  61,  59, ..., 162, 249, 254]], dtype=uint8)

In [60]:
current_image_index = 0
filename = ""
user_choices = {}
def show_image(face_data):
    global current_image_index, user_choices, filename
    #filename, faces = face_data
    # Initialize variables
    current_image_index = 0

    # Function to show the next image
    def show_next_image(face_data):
        global current_image_index
        if current_image_index < len(face_data):
            face = face_data[current_image_index][2]
            load_and_display_image(face)
            current_image_index += 1
        else:
            messagebox.showinfo("Finished","All images are done!")
            root.destroy()

    # Function to load and display an image
    def load_and_display_image(face):
        image1 = Image.fromarray(face)
        image2 = image1.resize((300, 300))  # Resize the image to fit the window
        photo = ImageTk.PhotoImage(image2)
        image_label.config(image=photo)
        image_label.image = photo  # Keep a reference to prevent garbage collection

    # Function to handle the "Yes" button
    def answer_yes():
        faceId = face_data[current_image_index-1][0]
        user_choices[faceId] = 1
        show_next_image(face_data)

    # Function to handle the "No" button
    def answer_no():
        faceId = face_data[current_image_index-1][0]
        user_choices[faceId] = 0
        show_next_image(face_data)

    # Create the main window
    root = tk.Tk()
    root.title("Image Viewer")

    # Create a label to display the images
    image_label = tk.Label(root)
    image_label.pack()

    # Create "Yes" and "No" buttons
    yes_button = tk.Button(root, text="Yes", command=answer_yes)
    no_button = tk.Button(root, text="No", command=answer_no)
    yes_button.pack(side=tk.LEFT)
    no_button.pack(side=tk.RIGHT)

    # Show the first image
    show_next_image(face_data)

    # Start the Tkinter main loop
    root.mainloop()

In [61]:

import numpy as np
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import os
show_image(face_data)

In [62]:
user_choices

{'face1': 1,
 'face2': 1,
 'face3': 1,
 'face4': 0,
 'face5': 0,
 'face6': 1,
 'face7': 0}

In [63]:

# Step 4: Append user choices to the existing dataframe
def append_user_choices(dataframe, user_choices):
    for faceId in user_choices:
        dataframe.loc[dataframe['FaceId'] == faceId, 'Consider'] = user_choices[faceId]
    return dataframe

# Step 4: Append user choices to the existing dataframe
df = append_user_choices(df, user_choices)
df

,FaceId,Filename,Face,Consider
0,face1,20161006_222700.jpg,"[[96, 80, 88, 83, 86, 89, 87, 75, 74, 83, 84, ...",1
1,face2,20161007_203010.jpg,"[[255, 254, 250, 255, 252, 255, 252, 222, 184,...",1
2,face3,20161007_203010.jpg,"[[174, 174, 174, 175, 175, 175, 175, 175, 176,...",1
3,face4,20161007_203010.jpg,"[[111, 85, 79, 76, 61, 65, 48, 45, 30, 33, 33,...",0
4,face5,20161007_203010.jpg,"[[185, 185, 184, 186, 186, 186, 186, 186, 185,...",0
5,face6,20161008_021946.jpg,"[[192, 194, 185, 185, 175, 148, 98, 36, 38, 39...",1
6,face7,20161008_021946.jpg,"[[25, 24, 26, 26, 27, 28, 29, 30, 30, 31, 27, ...",0


In [70]:
# Step 5: Prepare a convolution model for face recognition
def create_face_recognition_model(input_shape):
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(2, activation='softmax')  # Binary classification (consider or not)
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Step 5: Prepare a convolution model for face recognition

X = np.array(df['Face'].tolist())
y = np.array(df['Consider'].tolist())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
input_shape = X_train[0].shape
input_shape

C:\Users\ravik\AppData\Local\Temp\ipykernel_12872\912249143.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(df['Face'].tolist())


(312, 312)

In [88]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assume df is your DataFrame containing the data

# Convert the 'Face' column into NumPy arrays
X = df['Face'].tolist()

# Filter out empty arrays and get the maximum length for non-empty arrays
non_empty_X = [x for x in X if len(x) > 0]

# Determine the maximum sequence length (number of values in each list)
max_length = max(len(lst) for lst in non_empty_X)

# Pad the sequences to make them uniform in length
X_padded = pad_sequences(non_empty_X, maxlen=max_length, padding='post', dtype='float32')

# Assuming the 'Consider' column represents the labels
y = df['Consider'].values

# Create and train the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(max_length, len(X_padded[0]), 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Reshape the input data for CNN
X_padded = X_padded.reshape(X_padded.shape[0], X_padded.shape[1], X_padded.shape[2], 1)

# Train the model
model.fit(X_padded, y, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
1/1 [==============================] - 14s 14s/step - loss: 1.5955e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 284ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 317ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 288ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 295ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 304ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 288ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss:

In [89]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:


model = create_face_recognition_model(input_shape)

# Step 6: Pass the dataframe to the convolution model
model.fit(X_train, y_train, epochs=10)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")